In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.request import Request, urlopen
from urllib.parse import urlparse
from urllib.error import HTTPError, URLError
import requests
import logging
from socket import timeout
import re
from dotenv import load_dotenv
load_dotenv()
bearer_token = os.environ.get("BEARER_TOKEN")

In [2]:
politifact_fake = pd.read_csv('fakenewsnet/politifact_fake.csv')
filtered_p_fake = pd.read_csv('data/politifact_fake_with_scraped_text.csv')
politifact_real = pd.read_csv('fakenewsnet/politifact_real.csv')
filtered_p_real = pd.read_csv('data/politifact_real_with_scraped_text.csv')


In [3]:
# politifact_fake

In [5]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def process_url(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        html = urlopen(req, timeout = 10)
        html = html.read()
        return text_from_html(html)
    except timeout:
        print("connection timedout")
    except URLError as e:
        print(e)
    except HTTPError as err:
        if err.code == 410 or 404:
            print("permanently deleted or removed, url should be removed")
        else:
            raise

In [8]:
# politifact_fake['text'] = ''
for index, url in enumerate(tqdm(politifact_fake['news_url'])):
    if politifact_fake['text'].iloc[index] == '':
        if type(url) != float:
            if urlparse(url).scheme:
                if politifact_fake['text'].iloc[index] == '':
                    politifact_fake['text'].iloc[index] = process_url(url)

  0%|          | 0/432 [00:00<?, ?it/s]

HTTP Error 410: Gone
HTTP Error 404: Not Found
HTTP Error 410: Gone
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 410: Gone
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1124)>
HTTP Error 406: Not Acceptable
<urlopen error [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1124)>
HTTP Error 404: Not Found
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 404: Not Found
<urlopen error [Errno 61] Connection refused>
HTTP Error 410: Gone
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 406: Not Acceptable
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Err

In [9]:
# filtered_p_fake = politifact_fake[politifact_fake['text'] != '' ]
# filtered_p_fake = filtered_p_fake[filtered_p_fake['text'].notnull()]
# filtered_p_fake.to_csv('data/politifact_fake_with_scraped_text.csv')

In [4]:
filtered_p_fake

,Unnamed: 0,id,news_url,title,tweet_ids,text
0,3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,Home Advertis...
1,4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,Washington Sources ...
2,6,politifact13919,http://blogs.trendolizer.com/2015/01/new-york-...,New York Man Wanted For Infecting 240 Men And ...,NaN,Other Trendolizers... Adrenal...
3,7,politifact14795,https://web.archive.org/web/20171027105356/htt...,Saudi Arabia to Behead 6 School Girls for Bein...,923126512458616832\t923135295070990341\t923189...,success fail ...
4,8,politifact14328,https://web.archive.org/web/20170702174006/htt...,Malia Obama Fired From Cushy Internship At Spa...,880455776107679747\t880457763876462598\t880461...,success fail ...
...,...,...,...,...,...,...
228,419,politifact14169,https://web.archive.org/web/20170528095037/htt...,Rubio: “Rape Victims Should Be In Custody If T...,663538460104392704\t663757208031780864\t663764...,success fail ...
229,421,politifact14992,http://www.trainnews.info/2018/01/rep-paul-gos...,Rep. Paul Gosar Asks Capitol Police to Arrest ...,958425300144218112\t958428242528145409\t958428...,Home Business Online Business Bitcoin Revi...
230,422,politifact14158,https://web.archive.org/web/20170602190500/htt...,WORSE THAN HITLER! Trey Gowdy’s Son Found In A...,865933040492703745,success fail ...
231,427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?,954602090462146560\t954602093171609600\t954650...,...


In [3]:
from gensim.corpora import Dictionary

MIN_DF = 1 # minium document frequency
MAX_DF = 0.6 # maximum document frequency

In [4]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')

In [5]:
tokenized_texts = []
for index, text in tqdm(enumerate(filtered_p_fake['text'])):
    tokenized_texts.append(tokenizer.tokenize(text.lower()))
        
# tokenized_text = tokenizer.tokenize(sample_text.lower())
# print(tokenized_text)

In [6]:
dictionary = Dictionary(tokenized_texts) # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF, 
                           no_above=MAX_DF)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [9]:
import os
os.environ['MALLET_HOME'] = 'mallet-2.0.8'

In [18]:
from gensim.models.wrappers import LdaMallet

PATH_TO_MALLET = 'mallet-2.0.8/bin/mallet'
N_TOPICS = 10
N_ITERATIONS = 500

lda = LdaMallet(mallet_path=PATH_TO_MALLET,
                corpus=corpus,
                id2word=dictionary,
                num_topics=N_TOPICS,
                iterations=N_ITERATIONS)

In [22]:
for topic in range(N_TOPICS):
    words = lda.show_topic(topic, 10)
    topic_n_words = ' '.join([word[0] for word in words])
    print('Topic {}: {}'.format(str(topic), topic_n_words))

Topic 0: share click opens window twitter facebook 0 presidential google shares
Topic 1: 2017 reply 2 march pm 3 domain 1 don whoopi
Topic 2: federal public search government tweet information report 04 opinion law
Topic 3: trump president donald comments election log house world white obama
Topic 4: 2021 biden april comments 3 facebook 4 joe 30 politics
Topic 5: stats trending trendolizer video contact general trump sports site twitter
Topic 6: email source morning show video 70 smiley rickey contact advertisement
Topic 7: 2017 trump archive breaking internet web machine wayback data day
Topic 8: 2020 2021 2018 2019 february 2016 world april november december
Topic 9: trump politics entertainment health facebook policy popular world video 2021


In [6]:
# politifact_real['text'] = ''
for index, url in enumerate(tqdm(politifact_real['news_url'])):
    if politifact_real['text'].iloc[index] == '':
        if type(url) != float:
            if urlparse(url).scheme:
                if politifact_real['text'].iloc[index] == '':
                    politifact_real['text'].iloc[index] = process_url(url)

<urlopen error timed out>
HTTP Error 404: Not Found
HTTP Error 503: Service Unavailable
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
HTTP Error 404: Not Found
HTTP Error 406: Not Acceptable
connection timedout
HTTP Error 404: Not Found
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
<urlopen error [Errno -5] No address associated with hostname>
HTTP Error 404: Not Found
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1123)>
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
HTTP Error 404: Not Found
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [9]:
# filtered_p_real = politifact_real[politifact_real['text'] != '' ]
# filtered_p_real = politifact_real[politifact_real['text'].notnull()]
# filtered_p_real.to_csv('data/politifact_real_with_scraped_text.csv')

In [4]:
headers = {
    'Authorization': f'Bearer {bearer_token}'
}
count = 0
for index, tweet_ids in enumerate(tqdm(filtered_p_real['tweet_ids'])):
    if count < 1:
        count+=1
        tweet_id_list = tweet_ids.split()
        for tweet_id in tweet_id_list:
            if count < 2:
                print(tweet_id)
                print(f'https://api.twitter.com/2/tweets?ids={tweet_id}&tweet.fields=created_at&expansions=author_id&user.fields=created_at')
                response = requests.get(f'https://api.twitter.com/2/tweets?ids={tweet_id}&tweet.fields=created_at&expansions=author_id&user.fields=created_at',  headers=headers)
                print(response.json())
                count+=1

967132259869487105
https://api.twitter.com/2/tweets?ids=967132259869487105&tweet.fields=created_at&expansions=author_id&user.fields=created_at
{'data': [{'id': '967132259869487105', 'author_id': '868603454', 'created_at': '2018-02-23T20:21:13.000Z', 'text': '"According to a new survey by the National Federation of Independent Business, the number of small business owners ready to expand is at its highest levels in the survey’s 45-year history." #Smallbiz https://t.co/FCLMVMnPFq'}], 'includes': {'users': [{'username': 'OwensNFIB', 'created_at': '2012-10-08T20:07:02.000Z', 'id': '868603454', 'name': 'Charles Owens'}]}}



In [14]:
filtered_p_real = filtered_p_real[filtered_p_real['tweet_ids'].notnull()]


In [18]:



def getTweetAuthors(tweet_ids_raw):
    count = 0
    print(tweet_ids_raw)
    tweet_ids = re.split(r'\\t', tweet_ids_raw)
    for tweet_id in tweet_ids:
        if count < 1:
            response = requests.get('https://api.twitter.com/2/tweets?ids={tweet_id}&tweet.fields=created_at&expansions=author_id&user.fields=created_at',  headers)
            count+=1


In [22]:
getTweetAuthors(filtered_p_real.tweet_ids)

0      967132259869487105\t967164368768196609\t967215...
1      942953459\t8980098198\t16253717352\t1668513250...
4      89804710374154240\t91270460595109888\t96039619...
5      690248006399049728\t690254026663821312\t690276...
6      1262604762\t10969740933\t11182364398\t17507543...
                             ...                        
548    21096374968\t21096771824\t9413452992876544\t12...
549    817357495047979008\t817357627566985217\t817357...
550    634287923135909888\t946743411100536832\t946816...
552                                   188871706637647874
553             1002208963239337984\t1024651239697666048
Name: tweet_ids, Length: 366, dtype: object


TypeError: expected string or bytes-like object

513\t817384443065176064\t817384541467897856\t817384814051328000\t817385091525701632\t817385189223649280\t817385231716155392\t817385331645435904\t817385736852873216\t817385828469055489\t817386024745762816\t817386059097108480\t817387024479121409\t817387968872779776\t817388126482010112\t817388228458201089\t817388248720961537\t817388288445153280\t817388601382227968\t817388599373131776\t817388841040572418\t817388939996700676\t817388987367256064\t817389083425120256\t817389081575493632\t817389170419081216\t817389258877050880\t817389698545045504\t817389721894744070\t817389833169633284\t817390262779609088\t817390380295585792\t817390471567667201\t817390489083244544\t817390583278960640\t817390772211380224\t817391142249656324\t817391191356424192\t817391217470279680\t817391278665003008\t817391359602593795\t817391657859432448\t817391864437415936\t817391891838746624\t817392120197591040\t817392695501062146\t817392814803812352\t817392834517102592\t817392970370576389\t817393110149959681\t817393194828607